In [ ]:
%pylab inline

In [ ]:
import numpy as np

In [ ]:
import matplotlib.gridspec as gridspec
params = {'legend.fontsize':16,
          'figure.figsize': (8, 8),
         'axes.labelsize': 16,
         'axes.titlesize': 16,
         'xtick.labelsize':16,
         'ytick.labelsize': 16}
pylab.rcParams.update(params)

In [ ]:
import matplotlib.ticker as mtick
def decibel_formatter(unit='dB'):
    def format_value(value, pos=None):
        dB = 10*np.log10(value)
        return f'{dB:.0f} {unit}'
    return format_value

In [ ]:
[i for i in range(3,5)]

In [ ]:
def Mtx(mis,o):
    # solve S such (A+Z-2BR)S+B = 0 
    # mis : array [m(0),m(1),m(2)] 
    # o: order
    # return such  S(0)=1-2Sum_j S(j), S(j), j=1,...,o 
    o = int(o)
    assert o>=1, "o>=1 needed"
    A = np.zeros((o,o))
    Z = np.zeros((o,o))
    B = np.zeros((o,1))
    R = np.ones((1,o))
    #next A is a symmetric matrix... fill upper triangle included diag
    np.fill_diagonal(A, mis[0])
    Z[0,0] = mis[2]
    B[0,0] = mis[1]    
    if o>=2:
        A[0,1] = mis[1]
        #
        B[1,0] = mis[2]
#    if o>=3:
#        A[0,2] = mis[2]
#        A[1,2] = mis[1]
    
    if o>=3:
        for c in range(2,o):
            A[c-1,c]=mis[1]
            A[c-2,c]=mis[2]
        #
        B[1,0] = mis[2]
    
#    else:
        #print("not yet")
    #print("1) A\n",A)
    A = A + A.T - np.diag(np.diag(A))
    #print("2) A\n",A)
    M = A+Z-2*B*R
    S = np.linalg.solve(M,-B).flatten()
    S = np.insert(S,0,1-2*np.sum(S),axis=0)        
    return S
                     
def Kval(n,m,s):
    n = int(np.abs(n))
    #order
    om = len(m)-1
    os = len(s)-1
    #print(om,os)
    # m(-2) S(n-2) + m(-1) S(n-1)+ m(0) S(n) +  m(1) S(n+1) + m(2)S(n+2)
    # m(-j)=m(j), S(-j) = S(j)
    #symetric ms
    ms = np.hstack((np.flip(m[1:]),m))
    ss = np.hstack((np.flip(s[1:]),s))
    #print("ms: ",ms)
    #print("ss: ",ss)
    # [n-2,n-1,n,n+1,n+2] in [-o,...,-1,0,1,..,o]
    if n-om>os:
        return 0
    else:
        idx1 = np.arange(-os,os+1)
        idx2 = np.arange(n-om,n+om+1)
        #print(idx1,idx2)
        xy, x_ind, y_ind = np.intersect1d(idx1, idx2, return_indices=True)
        #print(xy,x_ind,y_ind)
        #print(ms[y_ind[0]:y_ind[-1]+1],ss[x_ind[0]:x_ind[-1]+1])
        return np.dot(ms[y_ind[0]:y_ind[-1]+1],ss[x_ind[0]:x_ind[-1]+1])

In [ ]:
mis=np.zeros(3)
#m_4
mis[0] = 2/3
mis[1] = 1/6

In [ ]:
mis

In [ ]:
for o in range(1,8):
    print(o,Mtx(mis,o))

In [ ]:
mis=np.zeros(3)
#m_6
mis[0] = 11/20
mis[1] = 13/60
mis[2] = 1/120

In [ ]:
for o in range(1,8):
    print(o,Mtx(mis,o))

In [ ]:
S= Mtx(mis,7)

In [ ]:
S

In [ ]:
mis

In [ ]:
Kval(0,mis,S)

In [ ]:
sum = 0
for n in range(0,11):
    kv = Kval(n,mis,S)
    if n==0:
        sum += kv
    else:
         sum += 2*kv
    print(n,Kval(n,mis,S))
print("Tot: ",sum)

In [ ]:
Kval(9,mis,S)

In [ ]:
Kval(10,mis,S)